In [ ]:
from keras import backend
from keras.engine.topology import Layer
import numpy as np

# https://github.com/kenoma/KerasFuzzy

class FuzzyLayer(Layer):

    def __init__(self,
                 output_dim,
                 initializer_centers=None,
                 initializer_sigmas=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'), )
        self.output_dim = output_dim
        self.initializer_centers = initializer_centers
        self.initializer_sigmas = initializer_sigmas
        super(FuzzyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_dimensions = list(input_shape)[:-1:-1]
        self.c = self.add_weight(name='c',
                                 shape=(input_shape[-1], self.output_dim),
                                 initializer=self.initializer_centers if not self.initializer_centers is None else 'uniform',
                                 trainable=True)
        self.a = self.add_weight(name='a',
                                 shape=(input_shape[-1], self.output_dim),
                                 initializer=self.initializer_sigmas if not self.initializer_sigmas is None else 'ones',
                                 trainable=True)
        super(FuzzyLayer, self).build(input_shape)

    def call(self, x):

        aligned_x = backend.repeat_elements(backend.expand_dims(x, axis=-1), self.output_dim, -1)
        aligned_c = self.c
        aligned_a = self.a
        for dim in self.input_dimensions:
            aligned_c = backend.repeat_elements(backend.expand_dims(aligned_c, 0), dim, 0)
            aligned_a = backend.repeat_elements(backend.expand_dims(aligned_a, 0), dim, 0)

        xc = backend.exp((-1) * backend.sum(backend.square((aligned_x - aligned_c) / (aligned_a << 1)), axis=-2, keepdims=False))
        # sums = K.sum(xc,axis=-1, keepdims=True)
        # less = backend.ones_like(sums) * backend.epsilon()
        return xc  # xc / backend.maximum(sums, less)

    def compute_output_shape(self, input_shape):
        return tuple(input_shape[:-1]) + (self.output_dim,)

Using TensorFlow backend.
